# Ciência dos Dados - Projeto 03 - 

#### Link para a base de dados já filtrada:
https://drive.google.com/open?id=1yq4DAJyJ2Er902X7Z1JuGc_t7w-aTIXJ

$Arthur\space Alegro \space de \space Oliveira$

$Pedro\space dos\space Santos \space e \space Silva$

$Jhonata\space Ferreira\space de \space Souza$

## Base de Dados:

Base de dados do ENEM referente aos anos de 2015 e 2016 extraídas pelo site http://portal.inep.gov.br/microdados.

Nomes dos arquivos:
* MICRODADOS_ENEM_2015.csv
* MICRODADOS_ENEM_2016.csv

**OBS.:** Para rodar corretamente este arquivo arquivo iPython Notebook (`.ipynb`)  deve-se extrair o arquivo `.zip` correspondente, acessar a pasta "DADOS" e colocar os arquivos `.csv` citados anteriormente no mesmo diretório deste arquivo.


## Importando Bibliotecas:

In [1]:
% matplotlib inline
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from math import *
import warnings
import time
import os

## Diretório:

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

## Verificando Dimensões dos arquivos:

In [ ]:
# def thousand_dot(number):
#     """Convert int value in a string with dots as thousands separator"""
#     lista = []
#     count = 0
#     for digit in (str(number))[::-1]:
#         lista.append(digit)
#         count +=1
#         if count == 3:
#             lista.append('.')
#             count = 0
#     lista.reverse
#     elements = ''.join(lista)[::-1]
#     return elements

# def dimension_calculator(filename, chunksize=1000, sep=','):
#     """Counts the amount of rowns and columns on a .csv file"""
#     # Raising warning:
#     warnings.warn("\n\nWARNING! Depending on chunksize value and/or file size this process might take time to compute")
#     # Parameters:
#     rows = 0
#     columns_labels = ''
#     count = True
#     # Process progress exibited to user:
#     print("Processing...")
#     # Opening database and counting values:
#     for chunk in pd.read_csv(filename, chunksize=chunksize, encoding='latin-1', header=0, sep=sep):
#         rows += len(chunk['NU_INSCRICAO'])
#         if count == True:
#             columns_labels = chunk.columns.values
#             count = False

#     # Process progress exibited to user:
#     print("Process complete\n")   
#     print("Number of rows: {}".format(thousand_dot(rows)))
#     print("Number of columns: {}".format(len(columns_labels)))
#     print("Number of elements: {}".format(thousand_dot(rows*len(columns_labels))))

* **2015**

In [ ]:
dimension_calculator("MICRODADOS_ENEM_2015.csv")

* **2016**

In [ ]:
dimension_calculator("MICRODADOS_ENEM_2016.csv", sep=';')

## Separando parte do DataFrame:

Selecionamos uma parte da base de dados (100.000 rows de dados) para análise em escala reduzida.

Após todas as filtragens e cálculos forem concluídos, todo o processo será refeito para cada `chunk`, podendo aplicar as implementações para todo a base de dados muito mais rápido, sem a necessidade de carregar o arquivo completo de uma vez (o que pode ser impossível, pois, em alguns casos, demanda mais processamento e memória que o computador possui)

def save_df(to_open_filename, to_save_filename, iteration_times, chunksize=1000):
    # Raising warning:
    warnings.warn("\n\nWARNING! Depending on chunksize value and/or file size this process might take time to compute")
    
    # Counters and Status Controllers:
    chunkcounter = 0
    shapecounter = [0, 0]
    add_header = True
    
    # Process progress exibited to user:
    print("Opening file...")
    
    # Loading command to export and append dataframes to a .xlsx file (also creates a new one, if it doesn't exist):
    writer = pd.ExcelWriter(to_save_filename, engine='openpyxl')
        
    # Loading database in chunks and defining chunk size, correct enconding and reading configs:
    for chunk in pd.read_csv(to_open_filename, chunksize=chunksize, encoding='latin-1', header=0, sep=','):
   
        if chunkcounter < iteration_times:
            # Selecting relevant parameters to ignore useless data:
            chunk = chunk[(chunk["TP_PRESENCA_CN"] == 1) & (chunk["TP_PRESENCA_CH"] == 1) & 
                          (chunk["TP_PRESENCA_LC"] == 1) & (chunk["TP_PRESENCA_MT"] == 1) & 
                          (chunk["TP_STATUS_REDACAO"] == 1) & (chunk["IN_TREINEIRO"] == 0)]
            
            # Selecting relevant parameters for fitering data:
            chunk = chunk.loc[:, ["SG_UF_RESIDENCIA", "NU_IDADE", "TP_ESTADO_CIIVL", "TP_COR_RACA", "TP_ST_CONCLUSAO", 
                                  "TP_ESCOLA", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "Q006"]]

            # Exporting dataframe to a new .xlsx file under existent data:
            chunk.to_excel(writer, sheet_name='Sheet1', index=False, startrow=shapecounter[0],startcol=0, header=add_header)
            
            # Counters update and progress of the process exhibited to the user:
            chunkcounter += 1
            shapecounter[0] += chunk.shape[0]
            shapecounter[1] = chunk.shape[1]
            print("Processing... ({}/{}) - File current size:({} rows, {} columns)".format(chunkcounter, iteration_times, shapecounter[0], shapecounter[1]))

        else:
            # Process progress exibited to user and breaking loop:
            print("Process complete")
            break
            
        # Counter update (avoid previous chunk's last row to be overwritten):
        if chunkcounter == 1:
            shapecounter[0] += 1
        # Status update (avoid header to be added again):
        add_header = False
        
    # Saving new .xlsx file:
    writer.save()

In [ ]:
# Creating filtered .xslx file:
save_df('MICRODADOS_ENEM_2015.csv','test8_dataframe.xlsx', chunksize=10000, iteration_times=775)

## Trabalhando com parte da Base de Dados:

In [ ]:
df = pd.read_excel('test8_dataframe.xlsx', encoding='latin-1', header=0)
df.shape()

### Separando em teste e treinamento:

In [ ]:
df_treinamento = df.loc[0.8*len(df), :] #Seria 80% do DataFrame para treinamento, e 20% para a base de Testes
df_teste = df.loc[0.2*len(df), :]

Nome das colunas:
* `SG_UF_RESIDENCIA`: Sigla da Unidade de Federação da residência do candidato
* `NU_IDADE`: Idade do candidato
* `TP_SEXO`: Sexo do candidato
* `TP_ESTADO_CIIVL`: Estado Civil do candidato
* `TP_COR_RACA`: Raça do candidato
* `TP_ST_CONCLUSAO`: Situação de conclusão do Ensino Médio
* `TP_ESCOLA`: Tipo de escola do Ensino Médio
* `NU_NOTA`: Notas nas provas, sendo: 

CN  | CH | LC | MT | REDACAO
------------- | ------------- | ------------- | ------------- | ------------- |
Ciências da Natureza  | Ciências Humanas | Linguagens e Códigos | Matemática | Redação
* Q006: Diz a renda mensal da família

# Porquê essas colunas foram escolhidas?

- A Unidade de Federação é bem relevante no acesso à escolaridade, pois sabe-se que, no Brasil, educação de boa qualidade, de maneira geral, é acessada por meios privados. Sendo assim, há regiões que são pobres em desenvolvimento humano.
- Idade influencia, tendo em vista que um jovem de 17-20 anos tende a lembrar mais do conteudo ante a um adulto de 30+
- "Raça" tem grande peso na nota, tendo em vista o sistema de cotas
- Situação de conclusão do Ensino Médio diz, aproximadamente, se o candidato se encontra preparado para a prova, ou não
- Tipo de escola nos ajuda a montar o perfil do aluno, e  o que esperar do seu desempenho
- Nota da prova é útil para a análise pra estabelecer a base de treinamento

# Análise descritiva:

- Gráficos de como cada variável afeta na nota do candidato
    - Boxplot de renda com nota
    - Gráfico de barras para nota com outra variável qualitativa
    

## Bibliografia:

Sites consultados:
* https://openpyxl.readthedocs.io/en/2.5/pandas.html
* http://portal.inep.gov.br/microdados
* 
* 
* 
* 
* 
* 